<a href="https://colab.research.google.com/github/radhakrishnan-omotec/hotel-repo/blob/main/Annem_Saad_Hotel_Project_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Facial Recognition for Personalized Hotel Guest Services

**Author**:
ANNEM SAAD

##**ABSTRACT**

Facial recognition technology has emerged as a transformative tool in various sectors, and its application in the hospitality industry promises to revolutionize guest experiences. This research explores the use of facial recognition for personalized hotel guest services, with the aim of streamlining check-in processes, enhancing security, and customizing guest services. The proposed system captures and processes guest images via a webcam interface, storing both facial images and personal information (such as name and room number) in a structured SQLite database. Facial feature detection, including the identification of eyes, smiles, and other key landmarks, is integrated to ensure high recognition accuracy even under varying conditions. Additionally, the system facilitates the linking of guest visual data with their associated profiles, allowing hotels to offer personalized services such as room preferences and tailored amenities based on historical data.

###Section 1: Importing Libraries

Here, we import the required libraries for webcam capture, data handling, GUI rendering, and database operations.

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import streamlit as st
import cv2
import os
import sqlite3


###Section 2: Webcam Image Capture Function

This section defines a function to capture an image using the webcam and save it to a specified location.

In [ ]:
# Function to capture and save an image using the webcam
def capture_image(save_path):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        st.error("Error: Could not open webcam")
        return

    st.write("Press 'Space' to Capture")

    while True:
        ret, frame = cap.read()
        if not ret:
            st.error("Error: Could not read frame")
            break

        cv2.imshow("Webcam Feed - Press space to capture", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord(" "):
            save_file = os.path.join(save_path, "captured_image.jpg")
            cv2.imwrite(save_file, frame)
            st.success(f"Image saved to {save_file}")
            break
        elif key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()


###Section 3: Database Connection Function

This section defines a function to establish a connection to the SQLite database.

In [ ]:
# Function to create a connection to SQLite database
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connected to database: {db_file}")
    except sqlite3.Error as e:
        print(e)
    return conn


###Section 4: Database Table Creation Function

Here, we define a function to create a table in the database if it doesn't already exist.

In [ ]:
# Function to create a table in the database if it doesn't already exist
def create_table(conn):
    try:
        query = '''CREATE TABLE IF NOT EXISTS user_data (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT NOT NULL,
                        age INTEGER,
                        email TEXT
                    );'''
        conn.execute(query)
        print("Table created successfully (if it didn't exist).")
    except sqlite3.Error as e:
        print(e)


###Section 5: Data Insertion into Database

This function handles inserting user data into the database.

In [ ]:
# Function to insert data into the database
def insert_data(conn, user_data):
    try:
        query = '''INSERT INTO user_data (name, age, email)
                   VALUES (?, ?, ?);'''
        conn.execute(query, user_data)
        conn.commit()  # Commit the transaction
        print("Data inserted successfully!")
    except sqlite3.Error as e:
        print(e)


###Section 6: Haar Cascade Classification

This section detects features such as faces, eyes, and smiles using Haar cascades.

In [ ]:
# Haar cascade classification function
def classify_and_save_features(face_cascade_path, eye_cascade_path, smile_cascade_path):
    # Check if cascade files exist
    for path in [face_cascade_path, eye_cascade_path, smile_cascade_path]:
        if not os.path.isfile(path):
            raise FileNotFoundError(f"Cascade file not found at {path}")

    # Load cascades
    face_cascade = cv2.CascadeClassifier(face_cascade_path)
    eye_cascade = cv2.CascadeClassifier(eye_cascade_path)
    smile_cascade = cv2.CascadeClassifier(smile_cascade_path)

    cap = cv2.VideoCapture(0)
    features_list = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]

            # Detect and mark eyes
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 10)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

            # Detect and mark smiles
            smiles = smile_cascade.detectMultiScale(roi_gray, 1.7, 22)
            for (sx, sy, sw, sh) in smiles:
                cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (0, 0, 255), 2)

        cv2.imshow("Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


###Section 7: Bill Generation Placeholder

This is a placeholder for the bill generation feature.

In [ ]:
# Placeholder for bill generation logic
def generate_bill(name):
    st.write(f"Bill generation logic for {name} goes here.")


###Section 8: Streamlit Main Application

This section integrates all functionalities into the Streamlit app.

In [ ]:
# Main function to run the Streamlit app
def main():
    st.title("Webcam Image Capture and Database Storage")

    if st.button("Click Image"):
        save_path = r"./captured_images"
        os.makedirs(save_path, exist_ok=True)
        capture_image(save_path)

    person_name = st.text_input("Write the Name of the Customer")
    if st.button("Store in Database"):
        database = "test.db"
        conn = create_connection(database)
        if conn:
            create_table(conn)
            age = st.number_input("Enter Age", min_value=0, max_value=120, step=1)
            email = st.text_input("Enter Email")
            if st.button("Insert Data"):
                insert_data(conn, (person_name, age, email))
            conn.close()

    if st.button("Classify"):
        classify_and_save_features('./haarcascade_frontalface_default.xml',
                                   './haarcascade_eye.xml',
                                   './haarcascade_smile.xml')

    name = st.text_input("Name for Bill Generation")
    if st.button("Generate Bill"):
        generate_bill(name)


###Section 9: Run the Application

Run the Streamlit app in this final section.

In [ ]:
if __name__ == "__main__":
    main()


# ENHANCEMENT # 1:Input Fields for Additional Guest Information

Add Input Fields for Additional Guest Information
Extend the form to collect more details during check-in, such as dietary preferences, arrival/departure times, and preferred languages.*italicized text*

# ENHANCEMENT # 2:Guest Visit Purpose and Preferences


Include input fields to capture the guest’s purpose of stay (e.g., business, vacation) and any specific preferences like quiet rooms or accessibility features. This helps staff better tailor their services.

# ENHANCEMENT # 3: Basic Billing Information Storage


Add input fields to capture expenses like meals, spa treatments, or minibar usage. This allows you to use the "Generate Bill" button to create a simple breakdown of costs, improving the guest checkout experience.



# ENHANCEMENT # 4:Notes Section

Include a free-text "Additional Notes" field where staff can record special requests or important details about the guest, such as anniversaries or specific instructions.



# ENHANCEMENT # 5:Confirmation Message After Data Entry

Display a clear confirmation message or summary after the data has been stored successfully, allowing staff to review the details and ensuring data entry accuracy.




